# LJ-Gradient: Numpy vs. Torch 

In [1]:
import numpy as np
import torch

In [2]:
def dist_torch(x): 
    """Calculates distance vectors and distances (euclidian norm of vecs)
    
    Arguments:
        x (float): position vectors (dim = N x 3)
    
    Output:
        dist (float): distances between particle pairs (dim = N x N)
        vecs (float): distance vectors between particle pairs (dim = N x N x 3)
    """
    x = torch.Tensor(x)
    vecs = x[None, :, :] - x[:, None, :]       
    return torch.norm(vecs, dim=-1), vecs

def gradLJ_t(x, sig=1, eps=1):
    dist, vecs = dist_torch(x)
    dist[dist!=0] = 1/dist[dist!=0]
    D_att = 6 * sig**6 * dist**8
    D_rep = -12 * sig**12 * dist**14
    D = 4*(eps*(D_att + D_rep))[:, :, None]*vecs
    return torch.sum(D, dim=-2)

# Felix
def vectors(coord, boxsize, pbc=False):
    vecs = coord[:, None, :] - coord[None, :, :]
    if not pbc:
        return vecs
    elif pbc:
        L = boxsize[1] - boxsize[0] #calculate boxlength
        vecs += (vecs<-0.5*L)*L - (vecs>0.5*L)*L
        return vecs

def distances(vectors):
    return np.linalg.norm(vectors,axis=-1)
    
def gradLJ(vecs, sig=1, eps=1):
    dist = distances(vecs)
    dist[dist!=0] = 1/dist[dist!=0]
    D_att = 6 * sig**6 * dist**8
    D_rep = -12 * sig**12 * dist**14
    D = 4*(eps*(D_att + D_rep))[:, :, None]*vecs
    return np.sum(D, axis=-2)

In [3]:
import random
import timeit

def Random_particles( N, dim=3 ):
    """Creates a list of N particles with random positions and charges
    
    Arguments:
        N (int): number of particles
        
    Output:
        x (float): position vectors (dim = N x 3)
        q (int): charges (dim = N)
    """
    return np.random.uniform(-1,1, size=(N,dim)), np.array([[-1, 1][random.randrange(2)] for i in range(N)])

In [4]:
x_init = Random_particles(100)[0]

In [5]:
%timeit gradLJ_t(x_init).numpy()
%timeit gradLJ(vectors(x_init, boxsize=(0,1)))

3.48 ms ± 171 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2.24 ms ± 21.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
